# 0.0. Imports

In [1]:
import pickle
import inflection

import numpy as np
import pandas as pd
import seaborn as sn

from sklearn import svm
from xgboost import XGBClassifier
from ydata_profiling import ProfileReport

from matplotlib import pyplot as plt

from sklearn import metrics as mt
from sklearn import ensemble as en
from sklearn import model_selection as ms
from sklearn import preprocessing   as pp

from sklearn import linear_model as lm
from sklearn import ensemble as en


/home/felipe/repos/hackday/hackday/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1.0. Data Description

## 1.1. Loading Data

In [2]:
df_raw = pd.read_csv('data/train.csv')
df_country = pd.read_csv('data/country_base_2.csv')

In [3]:
df1 = pd.merge(df_raw, df_country, on='Nacionalidade', how='left')

In [4]:
df1 = df1.drop('country', axis=1)

## 1.2. Rename Columns

In [5]:
cols_old = ['id', 'Classificacao_do_hotel', 'Meses_da_reserva_ate_o_check-in', 'Numero_de_pernoites_reservadas', 
            'Numero_de_hospedes', 'Regime_de_alimentacao', 'Nacionalidade', 'Forma_de_Reserva',
            'Ja_se_hospedou_anterioremente', 'Tipo_do_quarto_reservado', 'Reserva_feita_por_agencia_de_turismo', 
            'Reserva_feita_por_empresa', 'Reserva_com_Estacionamento', 'Reserva_com_Observacoes', 
            'Reserva_Cancelada', 'latitude', 'longitude']

snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))
df1.columns = cols_new

## 1.3. Data Dimensions

In [6]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of cols: {}'.format(df1.shape[1]))

Number of rows: 72159
Number of cols: 17


## 1.4. Check NA

In [7]:
df1.isna().sum()#/len(df1)

id                                         0
classificacao_do_hotel                     0
meses_da_reserva_ate_o_check_in            0
numero_de_pernoites_reservadas             0
numero_de_hospedes                         3
regime_de_alimentacao                      0
nacionalidade                           1093
forma_de_reserva                           0
ja_se_hospedou_anterioremente              0
tipo_do_quarto_reservado                   0
reserva_feita_por_agencia_de_turismo       0
reserva_feita_por_empresa                  0
reserva_com_estacionamento                 0
reserva_com_observacoes                    0
reserva_cancelada                          0
latitude                                1093
longitude                               1093
dtype: int64

### 1.4.1. Replace NA

In [8]:
#df1['numero_de_hospedes'] = df1['numero_de_hospedes'].fillna(2)
df1 = df1.dropna()

In [9]:
# import pandas as pd
# from sklearn.linear_model import LinearRegression
# from sklearn.impute import SimpleImputer

# # carregar dados de exemplo com valores ausentes
# df = pd.read_csv('exemplo.csv')

# # separar os dados em variáveis independentes e dependentes
# X = df.drop('Target', axis=1)
# y = df['Target']

# # criar um objeto SimpleImputer para imputar valores ausentes
# imputer = SimpleImputer(strategy='mean')

# # imputar os valores ausentes usando regressão linear
# for column in X.columns:
#     X_column = X[[column]]
#     X_column_with_nulls = X_column[X_column[column].isnull()]
#     X_column_without_nulls = X_column[X_column[column].notnull()]
#     y_without_nulls = y[X_column[column].notnull()]
#     model = LinearRegression()
#     model.fit(X_column_without_nulls, y_without_nulls)
#     y_pred = model.predict(X_column_with_nulls)
#     X_column_with_nulls[column] = y_pred
#     X[[column]] = pd.concat([X_column_with_nulls, X_column_without_nulls], axis=0)

# # juntar as variáveis independentes e dependentes de volta em um dataframe
# df_imputed = pd.concat([X, y], axis=1)

# # exibir o dataframe resultante
# print(df_imputed)

## 1.5. Data Dtypes

In [10]:
df1.dtypes

id                                        int64
classificacao_do_hotel                   object
meses_da_reserva_ate_o_check_in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                      float64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente            object
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo     object
reserva_feita_por_empresa                object
reserva_com_estacionamento               object
reserva_com_observacoes                  object
reserva_cancelada                         int64
latitude                                float64
longitude                               float64
dtype: object

### 1.5.1. Change Dtypes

In [11]:
df1['numero_de_hospedes'] = df1['numero_de_hospedes'].astype('int')

## 1.6. Data Balancing

In [12]:
df1['reserva_cancelada'].value_counts(normalize=True)

0    0.62674
1    0.37326
Name: reserva_cancelada, dtype: float64

## 1.7. Pandas Profiling

In [13]:
# prof = ProfileReport(df1)
# prof.to_file(output_file='output.html')
# # regime_de_alimentacao -> desbalanceado
# # nacionalidade -> desbalanceado
# # forma_de_reserva -> desbalanceado
# # ja_se_hospedou_anterioremente -> desbalanceado
# # tipo_do_quarto_reservado -> desbalanceado
# # reserva_feita_por_agencia_de_turismo -> desbalanceado
# # reserva_com_estacionamento -> desbalanceado

### 1.7.1. Meses da reserva até o check-in

In [14]:
df1[df1['meses_da_reserva_ate_o_check_in']>24].sample(10)

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada,latitude,longitude
54877,10526,5 estrelas,271,3,2,Sem refeicao,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
40073,87866,4 estrelas,84,4,4,Café da manha,France,Agência,Não,Tanzanite,Sim,Não,Não,1 a 3,0,46227638.0,2213749.0
16969,96638,4 estrelas,167,7,2,Café da manha,Norway,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,0,60472024.0,8468946.0
58812,12913,5 estrelas,270,12,2,Café da manha e jantar,Germany,Agência,Não,Pink Sapphire,Sim,Não,Não,1 a 3,1,51165691.0,10451526.0
26444,70919,4 estrelas,195,3,2,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
56104,38387,5 estrelas,175,2,2,Café da manha,United Kingdom,Balcão,Não,Amethyst,Sim,Não,Não,1 a 3,0,55378051.0,-3435973.0
64463,2099,5 estrelas,41,7,2,Café da manha,Romania,Agência,Não,Pink Sapphire,Sim,Não,Sim,1 a 3,0,45943161.0,2496676.0
48156,51116,4 estrelas,159,7,2,Café da manha,United Kingdom,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,0,55378051.0,-3435973.0
58897,13730,5 estrelas,196,3,2,Café da manha,Spain,Agência,Não,Amethyst,Não,Não,Não,Nenhuma,1,40463667.0,-374922.0
13381,25330,5 estrelas,178,0,2,Café da manha,France,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,0,46227638.0,2213749.0


In [15]:
df1[df1['meses_da_reserva_ate_o_check_in']>60][['id','reserva_cancelada']].groupby('reserva_cancelada').count().reset_index()

,reserva_cancelada,id
0,0,757
1,1,437


### 1.7.2. numero_de_pernoites_reservadas

In [16]:
df1[df1['numero_de_pernoites_reservadas']>365].sample(10)

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada,latitude,longitude
19772,72370,4 estrelas,12,475,623,Café da manha,Norway,Agência,Não,Green Emerald,Sim,Não,Não,Nenhuma,1,60472024.0,8468946.0
10807,84231,4 estrelas,2,1172,1,Café da manha,France,Balcão,Não,Amethyst,Não,Não,Não,1 a 3,0,46227638.0,2213749.0
17839,22621,5 estrelas,1,1024,3,Café da manha,Spain,Agência,Não,Green Emerald,Sim,Não,Não,1 a 3,0,40463667.0,-374922.0
37981,90924,4 estrelas,3,988,3,Café da manha e jantar,Italy,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,0,4187194.0,1256738.0
42419,4086,5 estrelas,5,1130,2,Café da manha e jantar,Spain,Agência,Não,Amethyst,Não,Não,Não,Nenhuma,1,40463667.0,-374922.0
24617,40530,4 estrelas,1,1117,3,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,0,40463667.0,-374922.0
51033,91463,4 estrelas,5,749,2,Café da manha,Switzerland,Balcão,Não,Amethyst,Não,Não,Não,1 a 3,0,46818188.0,8227512.0
44340,89134,4 estrelas,4,634,3,Café da manha,Austria,Agência,Não,Green Emerald,Sim,Não,Não,1 a 3,0,47516231.0,14550072.0
16918,59617,4 estrelas,8,775,2,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
67364,10678,5 estrelas,12,403,2,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0


In [17]:
df1[df1['numero_de_pernoites_reservadas']>365][['id','reserva_cancelada']].groupby('reserva_cancelada').count().reset_index()

,reserva_cancelada,id
0,0,660
1,1,375


In [18]:
df1[df1['numero_de_pernoites_reservadas']==0][['id','reserva_cancelada']].groupby('reserva_cancelada').count().reset_index()

,reserva_cancelada,id
0,0,391
1,1,12


### 1.7.3. numero_de_hospedes

In [19]:
df1[df1['numero_de_hospedes']>10]

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada,latitude,longitude
75,15778,5 estrelas,3,5,1020,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,0,40463667.0,-374922.0
102,2436,5 estrelas,11,7,20,Café da manha,Spain,Balcão,Não,Green Emerald,Não,Não,Não,Nenhuma,1,40463667.0,-374922.0
179,73042,4 estrelas,75,6,758,Café da manha,Poland,Agência,Não,Green Emerald,Sim,Não,Não,Nenhuma,1,51919438.0,19145136.0
187,81884,4 estrelas,2,3,470,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,0,40463667.0,-374922.0
251,83293,4 estrelas,1,2,381,Café da manha,Spain,B2B,Sim,Amethyst,Não,Sim,Não,1 a 3,0,40463667.0,-374922.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71702,52893,4 estrelas,1,1,269,Café da manha,Spain,B2B,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
71711,8940,5 estrelas,5,2,502,Café da manha,United Kingdom,Agência,Não,Green Emerald,Sim,Não,Não,1 a 3,1,55378051.0,-3435973.0
71848,20468,5 estrelas,1,1,166,Café da manha,Spain,Agência,Sim,Amethyst,Não,Sim,Sim,1 a 3,0,40463667.0,-374922.0
71975,43193,4 estrelas,2,1,780,Café da manha e jantar,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,0,40463667.0,-374922.0


In [20]:
df1[df1['numero_de_hospedes']>10][['id', 'reserva_feita_por_empresa']].groupby('reserva_feita_por_empresa').count().reset_index()


,reserva_feita_por_empresa,id
0,Não,1359
1,Sim,83


In [21]:
df1[df1['numero_de_hospedes']==0][['id', 'reserva_feita_por_agencia_de_turismo']].groupby('reserva_feita_por_agencia_de_turismo').count().reset_index()


,reserva_feita_por_agencia_de_turismo,id
0,Não,33
1,Sim,68


# 2.0. Feature Engineering

In [22]:
df2 = df1.copy()

In [23]:
df2['classificacao_do_hotel'] = df2['classificacao_do_hotel'].apply(lambda x: x.replace(' estrelas', ''))

In [24]:
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Sim', '1'))
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Não', '0'))


In [25]:
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Não', '0'))

In [26]:
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].apply(lambda x: x.replace('Não', '0'))

In [27]:
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].apply(lambda x: x.replace('Não', '0'))

In [28]:
# df2['regime_de_alimentacao'] = df2['regime_de_alimentacao'].apply(lambda x: x.replace('Café da manha e jantar', '1'))
# df2['regime_de_alimentacao'] = df2['regime_de_alimentacao'].apply(lambda x: x.replace('Café da manha, almoco e jantar', '4'))
# df2['regime_de_alimentacao'] = df2['regime_de_alimentacao'].apply(lambda x: x.replace('Café da manha', '2'))
# df2['regime_de_alimentacao'] = df2['regime_de_alimentacao'].apply(lambda x: x.replace('Sem refeicao', '3'))



In [29]:
# df2['forma_de_reserva'] = df2['forma_de_reserva'].apply(lambda x: x.replace('Agência', '1'))
# df2['forma_de_reserva'] = df2['forma_de_reserva'].apply(lambda x: x.replace('Balcão', '2'))
# df2['forma_de_reserva'] = df2['forma_de_reserva'].apply(lambda x: x.replace('B2B', '3'))


In [30]:
# df2['reserva_com_observacoes'] = df2['reserva_com_observacoes'].apply(lambda x: x.replace('Nenhuma', '1'))
# df2['reserva_com_observacoes'] = df2['reserva_com_observacoes'].apply(lambda x: x.replace('1 a 3', '2'))
# df2['reserva_com_observacoes'] = df2['reserva_com_observacoes'].apply(lambda x: x.replace('Mais de 3', '3'))


In [31]:
df2.head()

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada,latitude,longitude
0,33571,5,5,5,2,Café da manha e jantar,France,Agência,0,Amethyst,1,0,1,Nenhuma,0,46227638.0,2213749.0
1,82458,4,167,3,2,Café da manha,Spain,Agência,0,Amethyst,1,0,0,Nenhuma,1,40463667.0,-374922.0
2,94061,4,4,3,2,Café da manha,Belgium,Agência,0,Amethyst,1,0,0,1 a 3,0,50503887.0,4469936.0
3,75196,4,13,2,2,Café da manha,Spain,Agência,0,Amethyst,1,0,0,Nenhuma,1,40463667.0,-374922.0
4,82940,4,4,2,2,Café da manha,Spain,Agência,0,Amethyst,1,0,0,Nenhuma,1,40463667.0,-374922.0


In [32]:
df2.dtypes

id                                        int64
classificacao_do_hotel                   object
meses_da_reserva_ate_o_check_in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                        int64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente            object
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo     object
reserva_feita_por_empresa                object
reserva_com_estacionamento               object
reserva_com_observacoes                  object
reserva_cancelada                         int64
latitude                                float64
longitude                               float64
dtype: object

In [33]:
df2['classificacao_do_hotel'] = df2['classificacao_do_hotel'].astype('int')
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].astype('int')
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].astype('int')
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].astype('int')
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].astype('int')

# df2['regime_de_alimentacao'] = df2['regime_de_alimentacao'].astype('int')
# df2['forma_de_reserva'] = df2['forma_de_reserva'].astype('int')
# df2['reserva_com_observacoes'] = df2['reserva_com_observacoes'].astype('int')


In [34]:
df2['regime_de_alimentacao'].unique()

array(['Café da manha e jantar', 'Café da manha', 'Sem refeicao',
       'Café da manha, almoco e jantar'], dtype=object)

# 3.0. Data Filtering

In [35]:
df3 = df2.copy()

In [36]:
# reserva_feita_por_agencia_de_turismo reserva_feita_por_empresa
#df3 = df3.drop(['reserva_feita_por_agencia_de_turismo', 'reserva_feita_por_empresa'], axis=1)

# 4.0. Data Preparation 

In [37]:
#df4 = df3.drop('nacionalidade', axis=1).copy()
df4 = df3.copy()

In [38]:
# mms = pp.MinMaxScaler()
# df4['id'] = mms.fit_transform(df4[['id']].values)

# df4['classificacao_do_hotel'] = mms.fit_transform(df4[['classificacao_do_hotel']].values)

# df4['meses_da_reserva_ate_o_check_in'] = mms.fit_transform(df4[['meses_da_reserva_ate_o_check_in']].values)

# df4['numero_de_pernoites_reservadas'] = mms.fit_transform(df4[['numero_de_pernoites_reservadas']].values)

# df4['numero_de_hospedes'] = mms.fit_transform(df4[['numero_de_hospedes']].values)


In [39]:
#tipo_do_quarto_reservado
# criar um dicionário de frequência de cada categoria
freq_dict = df4['tipo_do_quarto_reservado'].value_counts(normalize=True).to_dict()
# aplicar o encoder ao dataframe
df4['tipo_do_quarto_reservado_encoded'] = df4['tipo_do_quarto_reservado'].map(freq_dict)

In [40]:
df4.head()

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada,latitude,longitude,tipo_do_quarto_reservado_encoded
0,33571,5,5,5,2,Café da manha e jantar,France,Agência,0,Amethyst,1,0,1,Nenhuma,0,46227638.0,2213749.0,0.718447
1,82458,4,167,3,2,Café da manha,Spain,Agência,0,Amethyst,1,0,0,Nenhuma,1,40463667.0,-374922.0,0.718447
2,94061,4,4,3,2,Café da manha,Belgium,Agência,0,Amethyst,1,0,0,1 a 3,0,50503887.0,4469936.0,0.718447
3,75196,4,13,2,2,Café da manha,Spain,Agência,0,Amethyst,1,0,0,Nenhuma,1,40463667.0,-374922.0,0.718447
4,82940,4,4,2,2,Café da manha,Spain,Agência,0,Amethyst,1,0,0,Nenhuma,1,40463667.0,-374922.0,0.718447


In [41]:
# dummy variables
cols_dummy = ['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes']

df4_dummy = pd.get_dummies(df4[cols_dummy])

df4 = pd.concat([df4, df4_dummy], axis=1)

# # numerical
# col_num = df4.select_dtypes(include=['int64', 'float64']).columns.tolist()

# # categorical
# col_cat = df4.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]']).columns.tolist()

# # encoding
# df4_dummy = pd.DataFrame(ohe.fit_transform(df4[col_cat]).toarray(), index=df4.index)
# pickle.dump(ohe, open('ohe.pkl', 'wb' ))

# # join numerical and categorical
# df42 = pd.concat([df4[col_num], df4_dummy], axis=1)
# df42.columns = df42.columns.astype(str)

# # one hot encoding
# ohe_class_hotel = pp.OneHotEncoder()
# ohe_aliment = pp.OneHotEncoder()
# ohe_form_reserv = pp.OneHotEncoder()
# ohe_hosp_anter = pp.OneHotEncoder()
# ohe_reserv_estac = pp.OneHotEncoder()
# ohe_reserva_observ = pp.OneHotEncoder()

# # classificacao_do_hotel
# ohe_class_hotel.fit(df4[['classificacao_do_hotel']])
# pickle.dump(ohe_class_hotel, open('ohe_class_hotel.pkl', 'wb' ))
# encoded = ohe_class_hotel.transform(df4[['classificacao_do_hotel']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_class_hotel.get_feature_names_out(['classificacao_do_hotel']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # regime_de_alimentacao
# ohe_aliment.fit(df4[['regime_de_alimentacao']])
# pickle.dump(ohe_aliment, open('ohe_aliment.pkl', 'wb' ))
# encoded = ohe_aliment.transform(df4[['regime_de_alimentacao']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_aliment.get_feature_names_out(['regime_de_alimentacao']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # forma_de_reserva
# ohe_form_reserv.fit(df4[['forma_de_reserva']])
# pickle.dump(ohe_form_reserv, open('ohe_form_reserv.pkl', 'wb' ))
# encoded = ohe_form_reserv.transform(df4[['forma_de_reserva']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_form_reserv.get_feature_names_out(['forma_de_reserva']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # ja_se_hospedou_anterioremente
# ohe_hosp_anter.fit(df4[['ja_se_hospedou_anterioremente']])
# pickle.dump(ohe_hosp_anter, open('ohe_hosp_anter.pkl', 'wb' ))
# encoded = ohe_hosp_anter.transform(df4[['ja_se_hospedou_anterioremente']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_hosp_anter.get_feature_names_out(['ja_se_hospedou_anterioremente']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # reserva_com_estacionamento
# ohe_reserv_estac.fit(df4[['reserva_com_estacionamento']])
# pickle.dump(ohe_reserv_estac, open('ohe_reserv_estac.pkl', 'wb' ))
# encoded = ohe_reserv_estac.transform(df4[['reserva_com_estacionamento']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_reserv_estac.get_feature_names_out(['reserva_com_estacionamento']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# #reserva_com_observacoes
# ohe_reserva_observ.fit(df4[['reserva_com_observacoes']])
# pickle.dump(ohe_reserva_observ, open('ohe_reserva_observ.pkl', 'wb' ))
# encoded = ohe_reserva_observ.transform(df4[['reserva_com_observacoes']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_reserva_observ.get_feature_names_out(['reserva_com_observacoes']))
# df4 = pd.concat([df4, df_encoded], axis=1)

In [42]:
# df4 = df4.drop(['tipo_do_quarto_reservado', 'classificacao_do_hotel', 'regime_de_alimentacao', 'forma_de_reserva',
#          'ja_se_hospedou_anterioremente', 'reserva_com_estacionamento', 'reserva_com_observacoes'], axis=1)


In [43]:
df4['latitude'] = df4['latitude'].apply(lambda x: np.radians(x/1000000))
df4['longitude'] = df4['longitude'].apply(lambda x: np.radians(x/1000000))

In [44]:
df4[['latitude', 'longitude']].sample(10)

,latitude,longitude
26429,0.706224,-0.006544
38094,0.706224,-0.006544
17296,0.073080,0.021934
28763,0.893010,0.182414
51690,0.893010,0.182414
8141,0.966529,-0.059969
43281,0.064735,-1.670505
11455,0.706224,-0.006544
56740,0.706224,-0.006544
6280,0.706224,-0.006544


# 5.0. Feature Selection 

In [45]:
df5 = df4.copy()

In [46]:
df5.head()

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,...,regime_de_alimentacao_Café da manha,regime_de_alimentacao_Café da manha e jantar,"regime_de_alimentacao_Café da manha, almoco e jantar",regime_de_alimentacao_Sem refeicao,forma_de_reserva_Agência,forma_de_reserva_B2B,forma_de_reserva_Balcão,reserva_com_observacoes_1 a 3,reserva_com_observacoes_Mais de 3,reserva_com_observacoes_Nenhuma
0,33571,5,5,5,2,Café da manha e jantar,France,Agência,0,Amethyst,...,0,1,0,0,1,0,0,0,0,1
1,82458,4,167,3,2,Café da manha,Spain,Agência,0,Amethyst,...,1,0,0,0,1,0,0,0,0,1
2,94061,4,4,3,2,Café da manha,Belgium,Agência,0,Amethyst,...,1,0,0,0,1,0,0,1,0,0
3,75196,4,13,2,2,Café da manha,Spain,Agência,0,Amethyst,...,1,0,0,0,1,0,0,0,0,1
4,82940,4,4,2,2,Café da manha,Spain,Agência,0,Amethyst,...,1,0,0,0,1,0,0,0,0,1


In [47]:
cols_drop =['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes', 'nacionalidade', 'tipo_do_quarto_reservado']
df5 = df4.drop(cols_drop, axis=1)

In [48]:
# split dataset into features and target
#X = df5.drop(['id', 'reserva_cancelada'], axis=1)
X = df5.drop(['reserva_cancelada'], axis=1)
y = df5['reserva_cancelada']

In [49]:
# split dataset into train and test
X_train, X_val, y_train, y_val = ms.train_test_split(X, y, test_size=0.2)

## 1.5. Data Dtypes

# 6.0. Machine Learning Model

## 6.1. XGBoost

In [50]:
# model definition
xgb_model = XGBClassifier(n_estimators=1000, max_depth=10, min_child_weight=2 )
#xgb_model = XGBClassifier()


# model training
xgb_model.fit(X_train,y_train.values.ravel())

# model prediction
#yhat = xgb_model.predict_proba(X_val)
yhat_xgb = xgb_model.predict(X_val)

In [51]:
# treshold = 0.8
# yhat = (yhat[:,1] >= treshold).astype(int)

In [52]:
# yhat = yhat[:,1]

# precision, recall, tresholds = mt.precision_recall_curve(y_val, yhat)

# plt.plot(recall, precision, marker='.', label='Model')

# plt.plot(tresholds, precision[:-1], 'b--', label='Precision')
# plt.plot(tresholds, recall[:-1], 'g-', label='Recall')


In [53]:
pickle.dump(xgb_model, open('/home/felipe/repos/hackday/xgb_model.pkl', 'wb' ))

### 6.1.1. Performance

In [54]:
precision = mt.precision_score(y_val, yhat_xgb)
recall = mt.recall_score(y_val, yhat_xgb)
f1_score = mt.f1_score(y_val, yhat_xgb, average='macro')

In [55]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.96282
Recall: 0.96428
F1 Score: 0.97104


## 6.2. Logistic Regression

In [56]:
# # model definition
# lr_model = lm.LogisticRegression()

# # model training
# lr_model.fit(X_train, y_train)

# # model prediction
# yhat_lr = lr_model.predict(X_val)


### 6.5.1. Performance

In [57]:
# precision = mt.precision_score(y_val, yhat_lr)
# recall = mt.recall_score(y_val, yhat_lr)
# f1_score = mt.f1_score(y_val, yhat_lr, average='macro')

In [58]:
# print('Precision: {:.5f}'.format(precision))
# print('Recall: {:.5f}'.format(recall))
# print('F1 Score: {:.5f}'.format(f1_score))

## 6.3. Gradiente Boosting

In [59]:
## model definition
gdr_model = en.GradientBoostingClassifier(n_estimators=500, random_state=42)

# model training
gdr_model.fit(X_train, y_train)

# model prediction
yhat_gb = gdr_model.predict(X_val)

In [60]:
pickle.dump(gdr_model, open('/home/felipe/repos/hackday/gdr_model.pkl', 'wb' ))

### 6.5.1. Performance

In [61]:
precision = mt.precision_score(y_val, yhat_gb)
recall = mt.recall_score(y_val, yhat_gb)
f1_score = mt.f1_score(y_val, yhat_gb, average='macro')

In [62]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.95327
Recall: 0.95744
F1 Score: 0.96450


## 6.4. Random Forest

In [63]:
# model definition
rf_model = en.RandomForestClassifier(n_estimators=1000, random_state=42, n_jobs=-1)

# model training
rf_model.fit(X_train, y_train)

# model prediction
yhat_rf = rf_model.predict(X_val)

In [64]:
pickle.dump(rf_model, open('/home/felipe/repos/hackday/rf_model.pkl', 'wb' ))

### 6.5.1. Performance

In [65]:
precision = mt.precision_score(y_val, yhat_rf)
recall = mt.recall_score(y_val, yhat_rf)
f1_score = mt.f1_score(y_val, yhat_rf, average='macro')

In [66]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.95840
Recall: 0.96314
F1 Score: 0.96880


## 6.5. SVC

In [67]:
# # model definition
# svc_model = svm.SVC()

# # model training
# svc_model.fit(X_train, y_train)

# # model prediction
# yhat_svc = svc_model.predict(X_val)

In [68]:
#pickle.dump(rf_model, open('/home/felipe/repos/hackday/rf_model.pkl', 'wb' ))

### 6.5.1. Performance

In [69]:
# precision = mt.precision_score(y_val, yhat_svc)
# recall = mt.recall_score(y_val, yhat_svc)
# f1_score = mt.f1_score(y_val, yhat_svc, average='macro')

In [70]:
# print('Precision: {:.5f}'.format(precision))
# print('Recall: {:.5f}'.format(recall))
# print('F1 Score: {:.5f}'.format(f1_score))

# 8.0. Ensemble

In [71]:
#ensemble = pd.DataFrame([yhat_xgb, yhat_lr, yhat_gb, yhat_rf, yhat_svc]).T
ensemble = pd.DataFrame([yhat_xgb, yhat_gb, yhat_rf]).T

In [72]:
#ensemble.columns = ['yhat_xgb', 'yhat_lr', 'yhat_gb', 'yhat_rf', 'yhat_svc']
ensemble.columns = ['yhat_xgb', 'yhat_gb', 'yhat_rf']

In [73]:
moda = ensemble.mode(axis=1)[0] # Encontra a moda das colunas e cria uma nova Series
ensemble['pred'] = moda # Adiciona a nova coluna ao DataFrame

### 6.5.1. Performance

In [74]:
yhat_en = ensemble['pred']

In [75]:
precision = mt.precision_score(y_val, yhat_en)
recall = mt.recall_score(y_val, yhat_en)
f1_score = mt.f1_score(y_val, yhat_en, average='macro')

In [76]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.96473
Recall: 0.96675
F1 Score: 0.97278
